In [ ]:
from tools.kernels import kernel, kernel_matrix

In [ ]:
x = [0, 1 ,2]
y = [1, 2, 1]

In [ ]:
kernel(x, y, type="bla")

In [ ]:
import numpy as np

In [ ]:
X = np.vstack((x, y))

In [ ]:
X

In [ ]:
kernel_matrix(X)